In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import EarlyStopping
from sklearn.cross_validation import train_test_split
import numpy as np
import time
import sys
import json
import time
import datetime
from keras.models import load_model

C:\Users\jmo4cob\AppData\Local\conda\conda\envs\tf35\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\jmo4cob\AppData\Local\conda\conda\envs\tf35\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def describe(X_shape, y_shape, batch_size, dropout, nb_epoch, conv_arch, dense):
    print (' X_train shape: ', X_shape) # (n_sample, 1, 48, 48)
    print (' y_train shape: ', y_shape) # (n_sample, n_categories)
    print ('      img size: ', X_shape[2], X_shape[3])
    print ('    batch size: ', batch_size)
    print ('      nb_epoch: ', nb_epoch)
    print ('       dropout: ', dropout)
    print ('conv architect: ', conv_arch)
    print ('neural network: ', dense)

In [3]:
conv_arch=[(32,3),(64,3),(128,3)]
dense=[64,2]
dropout=0.5
batch_size=128
nb_epoch=100
validation_split=0.2
patience=5
dirpath='model/'

In [4]:
def logging(model, starttime, batch_size, nb_epoch, conv_arch,dense, dropout,
            X_shape, y_shape, train_acc, val_acc, dirpath):
    now = time.ctime()
    model.save_weights('model/weights/{}'.format(now))
    save_model(model.to_json(), now, dirpath)
    save_config(model.get_config(), now, dirpath)
    save_result(starttime, batch_size, nb_epoch, conv_arch, dense, dropout,
                    X_shape, y_shape, train_acc, val_acc, dirpath)

In [5]:
# import dataset:
X_fname = 'dataset/X_train_data_Training.npy'
y_fname = 'dataset/y_train_data_Training.npy'
X_train = np.load(X_fname)
y_train = np.load(y_fname)


In [6]:
starttime = time.time()
X_train = X_train.astype('float32')
X_shape = X_train.shape
y_shape = y_train.shape
describe(X_shape, y_shape, batch_size, dropout, nb_epoch, conv_arch, dense)

 X_train shape:  (28709, 48, 48, 1)
 y_train shape:  (28709, 6)
      img size:  48 1
    batch size:  128
      nb_epoch:  100
       dropout:  0.5
conv architect:  [(32, 3), (64, 3), (128, 3)]
neural network:  [64, 2]


In [18]:
# train_tensors = X_train.astype('float32')/255
# valid_tensors = paths_to_tensor(valid_files).astype('float32')/255
# test_tensors = paths_to_tensor(test_files).astype('float32')/255

In [7]:
model = Sequential()
model.add(Conv2D(32, 3, strides=(1, 1), padding='same', data_format='channels_last', 
                    activation='relu', input_shape=(48, 48, 1)))
for i in range(conv_arch[0][1]-1):
    model.add(Conv2D(conv_arch[0][0], 3, strides=(1, 1), padding='same', data_format='channels_last', activation='relu' ))
model.add(MaxPooling2D(pool_size=(2, 2), padding='valid'))

for i in range(conv_arch[1][1]):
    model.add(Conv2D(conv_arch[1][0], 3, strides=(1, 1), padding='same', data_format='channels_last', activation='relu' ))
model.add(MaxPooling2D(pool_size=(2, 2), padding='valid'))

for i in range(conv_arch[2][1]):
    model.add(Conv2D(conv_arch[2][0], 3, strides=(1, 1), padding='valid', data_format='channels_last', activation='relu' ))
model.add(MaxPooling2D(pool_size=(2, 2), padding='valid'))

model.add(Flatten())

for i in range(dense[1]):
    model.add(Dense(dense[0], activation='relu'))
    model.add(Dropout(dropout))
    
model.add(Dense(y_train.shape[1], activation='softmax'))

In [8]:
# optimizer:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 48, 48, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 48, 48, 32)        9248      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 48, 48, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 24, 24, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 24, 24, 64)        36928     
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 24, 24, 64)        36928     
__________

In [10]:
# set callback:
callbacks = []
if patience != 0:
    early_stopping = EarlyStopping(monitor='val_loss', patience=patience, verbose=1)
    callbacks.append(early_stopping)

print ('Training....')
# fits the model on batches with real-time data augmentation:
# hist = model.fit_generator(datagen.flow(X_train, y_train, batch_size=batch_size),
#                 samples_per_epoch=len(X_train), nb_epoch=nb_epoch, validation_data=(X_test,y_test), callbacks=callbacks, verbose=1)

'''without data augmentation'''
hist = model.fit(X_train, y_train, nb_epoch=nb_epoch, batch_size=batch_size,
          validation_split=validation_split, callbacks=callbacks, shuffle=True, verbose=1)

Training....


C:\Users\jmo4cob\AppData\Local\conda\conda\envs\tf35\lib\site-packages\keras\models.py:848: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 22967 samples, validate on 5742 samples
Epoch 1/100
22967/22967 [==============================] - 40s - loss: 1.7768 - acc: 0.2453 - val_loss: 1.7218 - val_acc: 0.2428
Epoch 2/100
22967/22967 [==============================] - 36s - loss: 1.6974 - acc: 0.2663 - val_loss: 1.6543 - val_acc: 0.3116
Epoch 3/100
22967/22967 [==============================] - 36s - loss: 1.6128 - acc: 0.3232 - val_loss: 1.5988 - val_acc: 0.3248
Epoch 4/100
22967/22967 [==============================] - 36s - loss: 1.5575 - acc: 0.3404 - val_loss: 1.5091 - val_acc: 0.3631
Epoch 5/100
22967/22967 [==============================] - 36s - loss: 1.5003 - acc: 0.3714 - val_loss: 1.4537 - val_acc: 0.4039
Epoch 6/100
22967/22967 [==============================] - 36s - loss: 1.4405 - acc: 0.3964 - val_loss: 1.4185 - val_acc: 0.4256
Epoch 7/100
22967/22967 [==============================] - 36s - loss: 1.3910 - acc: 0.4196 - val_loss: 1.3430 - val_acc: 0.4345
Epoch 8/100
22967/22967 [=======================

In [11]:
# model result:
train_val_accuracy = hist.history
train_acc = train_val_accuracy['acc']
val_acc = train_val_accuracy['val_acc']
print ('          Done!')
print ('     Train acc: ', train_acc[-1])
print ('Validation acc: ', val_acc[-1])
print (' Overfit ratio: ', val_acc[-1]/train_acc[-1])

          Done!
     Train acc:  0.6413114468559409
Validation acc:  0.5470219434075803
 Overfit ratio:  0.8529739272382876


In [12]:
x = datetime.datetime.now()

In [13]:
model.save('model/'+'model_on_'+str(x)[:-16]+'.h5')

In [14]:
myModel = load_model('model/'+'model_on_'+str(x)[:-16]+'.h5')